In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

from sklearn.model_selection import StratifiedKFold
from sklearn import metrics, preprocessing
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from catboost import Pool, CatBoostClassifier

from keras import backend as K
from keras.layers import *
from keras.layers import Dense,Flatten,Dropout,Input,BatchNormalization
from keras.optimizers import RMSprop,Adam
from keras.utils import to_categorical
from keras.models import Model
import tensorflow as tf
import os
from dfencoder import AutoEncoder
import lightgbm as lgb


# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#!pip install dfencoder

In [5]:
#Importing
train_data = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/training_v2.csv')
test_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/unlabeled.csv')

sub_data =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/samplesubmission.csv')
data_dict = pd.read_csv('/Users/s0c02nj/Desktop/WiDS/WiDS Datathon 2020 Dictionary.csv')

In [6]:
#x_train = train_data.drop('hospital_death',axis=1)
y_train = train_data['hospital_death']

In [ ]:
#x_test = test_data.drop('hospital_death',axis=1)

In [ ]:
#x_comb = pd.concat([x_train,x_test],sort=False)

In [ ]:
#x_train.index = range(0,len(x_train))

In [ ]:
#x1_train = x_train.sample(n = int(x_train.shape[0]*0.65))

In [ ]:
#x1_val = x_train.loc[~x_train.index.isin(x1_train.index)]

In [ ]:
# x1_train, x1_val, y1_train, y1_val = train_test_split(x_train, y_train, 
#                                                       test_size=0.2,
#                                                       random_state=0,
#                                                       stratify = y_train)

In [3]:
df_train = np.load('/Users/s0c02nj/Downloads/train_features.npy')
df_test = np.load('/Users/s0c02nj/Downloads/test_features.npy')
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [4]:
df_train = pd.DataFrame(df_train)
df_test = pd.DataFrame(df_test)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=50)

In [ ]:
pca.fit(df_tr)

In [ ]:
print(pca.explained_variance_ratio_.sum())

In [ ]:
df_tr_pca = pca.transform(df_tr)
df_te_pca = pca.transform(df_te)

In [ ]:
df_tr_pca = pd.DataFrame(df_tr_pca)
df_te_pca = pd.DataFrame(df_te_pca)

In [ ]:
#df_tr_pca.to_csv('/Users/s0c02nj/Desktop/WiDS/deep_tr_feat.csv',index = False)


In [ ]:
#df_te_pca.to_csv('/Users/s0c02nj/Desktop/WiDS/df_te_pca.csv',index = False)

In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(df_train, y_train, 
                                                      test_size=0.2,
                                                      random_state=0,
                                                      stratify = y_train)

In [8]:
random_state = 42
np.random.seed(random_state)

lgb_params = {
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth':8,
    'metric':{'auc'},
    'num_threads': -1,
    'objective': 'binary',
    'verbosity': 1,
    'num_leaves':15,
    'bagging_fraction':0.5, #rows
    'bagging_frequency':5,
    'min_data_in_leaf':50,
    'feature_fraction':0.2, # features
    'lambda_l1':10,
    'lambda_l2':15,
    'bagging_seed':2019,
    'min_gain_split':0.3
}

In [9]:
trn_data = lgb.Dataset(x1_train, label= y1_train)
val_data = lgb.Dataset(x1_val,  label=  y1_val)

In [10]:
lgb_clf = lgb.train(lgb_params,
                    trn_data,
                    13000,
                    valid_sets = [trn_data, val_data],
                    early_stopping_rounds=3000,
                    verbose_eval=200
                    )

Training until validation scores don't improve for 3000 rounds.
[200]	training's auc: 0.885492	valid_1's auc: 0.868255
[400]	training's auc: 0.896213	valid_1's auc: 0.874855
[600]	training's auc: 0.904162	valid_1's auc: 0.878488
[800]	training's auc: 0.910415	valid_1's auc: 0.880799
[1000]	training's auc: 0.915802	valid_1's auc: 0.882342
[1200]	training's auc: 0.92047	valid_1's auc: 0.883537
[1400]	training's auc: 0.924828	valid_1's auc: 0.884356
[1600]	training's auc: 0.928783	valid_1's auc: 0.88494
[1800]	training's auc: 0.932416	valid_1's auc: 0.885292
[2000]	training's auc: 0.935805	valid_1's auc: 0.885706
[2200]	training's auc: 0.938976	valid_1's auc: 0.886077
[2400]	training's auc: 0.941987	valid_1's auc: 0.886282
[2600]	training's auc: 0.944883	valid_1's auc: 0.886526
[2800]	training's auc: 0.947652	valid_1's auc: 0.886651
[3000]	training's auc: 0.950276	valid_1's auc: 0.886758
[3200]	training's auc: 0.952722	valid_1's auc: 0.886816
[3400]	training's auc: 0.955076	valid_1's auc:

KeyboardInterrupt: 

In [ ]:
#df_te.shape

In [ ]:
test_prob = model.predict(df_te)

In [ ]:
pred = test_prob[:,1]

In [ ]:
data_sub = x_test[['encounter_id']]
data_sub['hospital_death'] = pred

In [ ]:
data_sub.to_csv('/Users/s0c02nj/Desktop/WiDS/pred_autoencoder.csv',index = False)

In [ ]:
a1 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/sub_10fold_keep_missing_as_it_is_10thFeb2020_lgb_type4_ff_20_na_bmi.csv')
a2 =  pd.read_csv('/Users/s0c02nj/Desktop/WiDS/pred_autoencoder.csv')

In [ ]:
sub = pd.DataFrame()
sub['encounter_id'] = a1['encounter_id']

In [ ]:
pred_ens = (a1['hospital_death'] + a2['hospital_death']
           )/2.0
            

In [ ]:
sub['hospital_death'] = pred_ens

In [ ]:
sub.to_csv('/Users/s0c02nj/Desktop/WiDS/ensemble_deep_lgb_14thfeb.csv',index = False)